In [1]:
import tsa_utils

from shutil import copytree
import numpy as np
import matplotlib as mpl
import os
import matplotlib.pyplot as plt
import matplotlib.animation
import tensorflow as tf
from numpy import genfromtxt
import pandas as pd
import sys
sys.path.append(os.getcwd())
import pip
pip.main(['install', '--upgrade', 'dask'])
import sklearn.decomposition
from sklearn.preprocessing import StandardScaler
import pickle
import scipy.misc
import skimage.io
import cv2
import skimage.measure
import pickle

Requirement already up-to-date: dask in /home/cody/anaconda3/envs/p2.7/lib/python2.7/site-packages


/home/cody/anaconda3/envs/p2.7/lib/python2.7/site-packages/pip/_vendor/requests/packages/urllib3/util/ssl_.py:318: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#snimissingwarning.
  SNIMissingWarning
/home/cody/anaconda3/envs/p2.7/lib/python2.7/site-packages/pip/_vendor/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#insecureplatformwarning.
  Insec

In [2]:
#Model parameters
BATCH_SIZE=10
FILTER_COUNT=12
KERNEL_SIZE1=(1,6,6)
DEPTHSTRIDE=1
XSTRIDE=1
YSTRIDE=1
POOLSIZE1=(16,3,3)
POOLSIZE2=(3,3)
POOL_STRIDES1=(1,2,2)
POOL_STRIDES=(1,1)
STEPS=3000
#XSIZE=392
XSIZE=512
#XSIZE=270
#YSIZE=340
YSIZE=660
LEARNING_RATE=0.01
EPSILON=0
IMAGE_DEPTH=16
CHANNELS=1
FLAT_POOL_SIZE=46656

DATA_PATH="./images/"
CHECKPOINT_PATH="/output/multi_class"
MODEL_ID="multi_class"

tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
class ZoneModel():

    def __init__(self, model_id, ids, x_slice, y_slice, data_path, labels, pca_model, checkpoint_path=".", localize=False):
        self.model_id = model_id
        self.ids = ids
        self.x_slice = x_slice
        self.y_slice = y_slice
        self.data_path = data_path
        self.checkpoint_path = checkpoint_path
        self.labels = labels
        self.localize = localize
        self.pca_model=pca_model

    def build_model(self, data, labels, mode):
        if mode == tf.contrib.learn.ModeKeys.INFER:
            BATCH_SIZE=1
        else:
            BATCH_SIZE=10
        print(XSIZE, YSIZE)
        data = tf.reshape(data, [BATCH_SIZE, IMAGE_DEPTH, YSIZE, XSIZE, CHANNELS])
        pool1 = tf.layers.max_pooling3d(inputs=data, pool_size=POOLSIZE1, strides=POOL_STRIDES1, name="pool1")
        flat_pool = tf.reshape(pool1, [BATCH_SIZE, FLAT_POOL_SIZE])
        flat_pool=tf.identity(flat_pool, name="flat_pool")

        logits = tf.layers.dense(inputs=flat_pool, units=17, trainable=True)
        logits = tf.identity(logits, name="logits")
        logits = tf.reshape(logits, [BATCH_SIZE,17])
        predictions = {
            "classes": tf.argmax(
              input=logits, axis=1, name="classes"),
          "probabilities": tf.nn.sigmoid(
              logits, name="prob_tensor")}

        if mode == tf.contrib.learn.ModeKeys.TRAIN:
            #flat_labels = tf.reshape(labels, [BATCH_SIZE, 17])
            labels=tf.identity(labels, name="labels")
            #class_weights=tf.reduce_sum(tf.multiply(flat_labels, tf.constant(WEIGHTS, dtype=tf.int64)), axis=1)
            #print(class_weights)
            #loss = tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=logits)
            loss = tf.losses.sigmoid_cross_entropy(labels, logits)
            train_op = tf.contrib.layers.optimize_loss(
                loss=loss,
                global_step=tf.contrib.framework.get_global_step(),
                learning_rate=LEARNING_RATE,
                optimizer="SGD")
        if mode == tf.contrib.learn.ModeKeys.INFER:
            return tf.contrib.learn.ModelFnOps(mode=mode, predictions=predictions)
        return tf.contrib.learn.ModelFnOps(mode=mode, predictions=predictions, loss=loss, train_op=train_op)

    def train_model(self, tensors_to_log, reuse=False):
        if reuse:
            tsa_classifier = self.model
        else:
            tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                     model_dir=self.checkpoint_path + "/" + self.model_id)
        logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)
        tsa_classifier.fit(
            x=tsa_utils.ResidueIterator(self.ids, self.data_path, self.pca_model), 
            y=tsa_utils.InputLabelsIterator(self.ids, self.labels), 
            steps=STEPS, 
            batch_size=BATCH_SIZE, 
            monitors=[logging_hook])
        self.model = tsa_classifier

    def load_model(self):
        print(self.checkpoint_path, self.model_id)
        tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                    model_dir=self.checkpoint_path + "/" + self.model_id)
        print ("LOADED MODEL AT STEP " + str(tsa_classifier.get_variable_value("global_step")) + " FROM " + self.checkpoint_path + "/" + self.model_id )
        self.model = tsa_classifier

    def bootstrap_model(self, path=None):
        if path:
            checkpoint_path = path
        else:
            checkpoint_path = self.checkpoint_path + "/" + self.model_id
        print ("USING CHECKPOINT PATH  " + checkpoint_path)
        tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                    model_dir=checkpoint_path)
        print ("BOOTSTRAPPED AT STEP " + str(tsa_classifier.get_variable_value("global_step")))
        self.model = tsa_classifier 

    def predict(self):
        return self.model.predict(x=tsa_utils.ResidueIterator(self.ids, 
                                                            self.data_path,
                                                            self.pca_model,
                                                            10000, 
                                                            self.y_slice,
                                                            self.x_slice, False))

In [4]:
image_df = pd.read_csv(DATA_PATH + '/stage1_labels.csv')
image_df['zone'] = image_df['Id'].str.split("_Zone", expand=True)[1].str.strip()
image_df['id'] = image_df['Id'].str.split("_", expand=True)[0].str.strip()

ids = image_df["id"].unique()
ids.sort()

labels = image_df[image_df['id'].isin(ids)]
labels = labels.sort_values("id")

tensors_to_log =  {"probabilities": "prob_tensor",
                    "actual":"labels"}
labels.zone = labels.zone.astype("int")
labels = labels.sort_values(["id","zone"])
labels = labels["Probability"]
labels = np.reshape(labels, (-1,17))

/home/cody/anaconda3/envs/p2.7/lib/python2.7/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


In [5]:
with tf.Session() as sess:
    model = ZoneModel(MODEL_ID, ids, "trimmed", "both", DATA_PATH, labels, CHECKPOINT_PATH, localize=False)     
    model.train_model(tensors_to_log, reuse=False)

INFO:tensorflow:Using default config.
Using default config.
INFO:tensorflow:Using config: {'_model_dir': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb7efe50>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}
Using config: {'_model_dir': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb7efe50>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_num_worke

From <ipython-input-3-e10f8de83c9f>:62: calling fit (from tensorflow.contrib.learn.python.learn.estimators.estimator) with y is deprecated and will be removed after 2016-12-01.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


From <ipython-input-3-e10f8de83c9f>:62: calling fit (from tensorflow.contrib.learn.python.learn.estimators.estimator) with x is deprecated and will be removed after 2016-12-01.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


From <ipython-input-3-e10f8de83c9f>:62: calling fit (from tensorflow.contrib.learn.python.learn.estimators.estimator) with batch_size is deprecated and will be removed after 2016-12-01.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


RETURNING RESIDUE
(16, 660, 512)
(512, 660)


ValueError: Cannot reshape a tensor with 838950 elements to shape [10,46656] (466560 elements) for 'Reshape_1' (op: 'Reshape') with input shapes: [10,1,329,255,1], [2] and with input tensors computed as partial shapes: input[1] = [10,46656].